# Simulate a temperature controller with an EPICS `swait` record

Learn how to create a simulated [temperature
controller](https://bcda-aps.github.io/bluesky_training/instrument/describe_instrument.html#temperature)
with Bluesky and an EPICS
[swait](https://bcda-aps.github.io/apstools/1.6.17/api/synApps/_swait.html)
record.  We'll show how to simulate the controller in EPICS and use that
simulation as a *positioner* in Bluesky.

In this simulation, the `swait` record provides the computations for the
feedback loop that updates the simulated temperature.

## Connect with a `swait` record

We'll connect with the `gp:userCalc18` PV, an instance of an EPICS `swait`
record in our example IOC.  We'll create the ophyd controller object using the
[SwaitRecord](https://bcda-aps.github.io/apstools/latest/api/synApps/_swait.html#apstools.synApps.swait.SwaitRecord)
structure from the [apstools](https://bcda-aps.github.io/apstools/latest/)
package.

In [1]:
from apstools.synApps import SwaitRecord

controller = SwaitRecord("gp:userCalc18", name="simulator")
controller.wait_for_connection()
print(f"{controller.read()=}\n{controller.read_configuration()=}")

controller.read()=OrderedDict([('simulator_calculated_value', {'value': 39.97393759060044, 'timestamp': 1703457763.712367})])
controller.read_configuration()=OrderedDict([('simulator_description', {'value': 'temperature', 'timestamp': 1703457763.712367}), ('simulator_scanning_rate', {'value': 6, 'timestamp': 1703457763.712367}), ('simulator_disable_value', {'value': 0, 'timestamp': 1703457763.712367}), ('simulator_scan_disable_input_link_value', {'value': 1, 'timestamp': 1703457763.712367}), ('simulator_scan_disable_value_input_link', {'value': 'gp:userCalcEnable.VAL CA MS', 'timestamp': 1703457763.712367}), ('simulator_forward_link', {'value': '0', 'timestamp': 1703457763.712367}), ('simulator_device_type', {'value': 0, 'timestamp': 1703457763.712367}), ('simulator_alarm_status', {'value': 0, 'timestamp': 1703457763.712367}), ('simulator_alarm_severity', {'value': 0, 'timestamp': 1703457763.712367}), ('simulator_new_alarm_status', {'value': 0, 'timestamp': 1703457763.712367}), ('simul

## Create a function to setup the controller

Create a function to configure the `swait` record as a simulated temperature
controller.  The "controller" will update the current computed value (the
*readback*) at `period` based on the setpoint.  Note that `period` here is one
of the preset EPICS `.SCAN` field values.  Pick from any of these values (from
the table at [this
reference](https://epics-base.github.io/epics-base/menuScan.html)):

- `"10 second"`
- `"5 second"`
- `"2 second"`
- `"1 second"`
- `".5 second"`
- `".2 second"`
- `".1 second"`

Be certain to use the exact text string as shown.

The `swait` record will compute the step size based on the difference between
the previous value and the setpoint, limited to the maximum step size.  A noise
factor is applied to each new computation.  The fields of the `swait` record in
this simulation are described in the next table:

field | description
--- | ---
`.VAL` | readback
`.B` | setpoint
`.A` | previous value
`.C` | noise factor
`.D` | maximum step size
`.CALC` | calculation expression
`.SCAN` | record scan `period`

The calculation will simulate a feedback loop which reduces the
value of `abs(readback - setpoint)`.

In [2]:
def setup_controller(
    swait,
    setpoint=None,
    label="controller",
    noise=2,
    period="1 second",
    max_change=2
):
    swait.reset()  # remove any prior configuration
    swait.description.put(label)
    swait.channels.A.input_pv.put(swait.calculated_value.pvname)
    if setpoint is not None:
        swait.channels.A.input_value.put(setpoint)  # readback
        swait.channels.B.input_value.put(setpoint)  # setpoint
    swait.channels.C.input_value.put(noise)
    swait.channels.D.input_value.put(max_change)
    swait.scanning_rate.put(period)
    swait.calculation.put("A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)")

## Setup our controller

Setup our controller with a (randomly-selected) setpoint and scan period. Watch
it start for a short time.  Gradually, the readback will approach the setpoint
value.

In [3]:
import random
import time

setup_controller(controller, 10 + 30 * random.random(), period="1 second", label="temperature")

t0 = time.time()
for i in range(5):
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={controller.calculated_value.get():.2f}"
        f" setpoint={controller.channels.B.input_value.get():.2f}"
    )

1.00s: readback=1.16 setpoint=18.73
2.00s: readback=2.77 setpoint=18.73
3.01s: readback=5.29 setpoint=18.73
4.01s: readback=6.56 setpoint=18.73
5.02s: readback=9.45 setpoint=18.73


## temperature as a positioner

A *positioner* is a device that has both a *readback* (the current measured value) and a *setpoint* (the expected, or demanded, value of the device).  These are available as EPICS PVs from our `swait` record.  We can obtain these directly from our ophyd `controller` object:

signal | swait field | ophyd object
--- | --- | ---
readback | `.VAL` | `controller.calculated_value.pvname`
setpoint | `.B` | `controller.channels.B.input_value.pvname`

We'll create the ophyd `temperature` positioner object using the
[PVPositionerSoftDoneWithStop](https://bcda-aps.github.io/apstools/latest/api/_devices.html#apstools.devices.positioner_soft_done.PVPositionerSoftDoneWithStop)
structure from the [apstools](https://bcda-aps.github.io/apstools/latest/)
package.

In [4]:
from apstools.devices import PVPositionerSoftDoneWithStop

temperature = PVPositionerSoftDoneWithStop(
    "",
    name="temperature",
    readback_pv=controller.calculated_value.pvname,
    setpoint_pv=controller.channels.B.input_value.pvname,
    tolerance=1,
)
temperature.wait_for_connection()
print(f"{temperature.position=}")

temperature.position=9.453620202944991


### Change the setpoint

Watch the readback after the setpoint is changed, until the temperature becomes
`inposition` (`inposition` is a property that reports a `True`/`False` value
determined by `abs(readback - setpoint) <= tolerance`).

Here, we lower the temperature setpoint by 10 from the current readback value.
Then, monitor the readback value until `inposition`.

In [5]:
temperature.setpoint.put(temperature.readback.get() - 10)

t0 = time.time()
while not temperature.inposition:
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={temperature.readback.get():.2f}"
        f" setpoint={temperature.setpoint.get():.2f}"
    )

1.00s: readback=7.80 setpoint=-0.55
2.01s: readback=6.23 setpoint=-0.55
3.01s: readback=1.88 setpoint=-0.55
4.02s: readback=0.63 setpoint=-0.55
5.02s: readback=-0.22 setpoint=-0.55


### Move the temperature as a positioner

Here, we treat the `temperature` object as a *positioner*.  

<b>Tip</b>:
In ophyd, a positioner object has a `move()` method and a `position` property.  The `position` property is a shortcut for `readback.get()`.

In [6]:
temperature.position

-0.21637293049515516

Set the `temperature` to `25` and wait for the *move* to complete.  A `MoveStatus` object is returned by the `move()` method.

<b>Tip</b>: Python prints the value of the last object shown.  In this case,
Python prints the value of the `MoveStatus` object.  It shows
that that the move is done, how long it took, whether the move was successful,
and other information.

In [7]:
temperature.move(25)

MoveStatus(done=True, pos=temperature, elapsed=13.0, success=True, settle_time=0.0)

Make a move *relative* to the current (**readback**) position:

In [8]:
temperature.move(temperature.position + 5)

MoveStatus(done=True, pos=temperature, elapsed=3.0, success=True, settle_time=0.0)

Make a move *relative* to the current **setpoint**:

In [9]:
temperature.move(temperature.setpoint.get() - 5)

MoveStatus(done=True, pos=temperature, elapsed=3.0, success=True, settle_time=0.0)

## Use the `temperature` positioner with a bluesky plan

The `temperature` positioner may be used as a detector or a positioner in a bluesky plan.

First, setup the bluesky objects needed for scanning and reporting.  We won't
need plots nor will we need to save any data.  Also create a convenience function to report the current parameters of the positioner.

In [10]:
from bluesky.run_engine import RunEngine
from bluesky import plans as bp
from bluesky import plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

def print_position(pos):
    print(
        f"inposition={pos.inposition}"
        f"  position={pos.position:.3f}"
        f"  setpoint={pos.setpoint.get():.3f}"
    )

Set the temperature to `25` using a bluesky plan stub (`bps.mv()`).  Here, `bps.mv()` will set the temperature to an *absolute* value.

A plan stub can be used directly with the `RE()` as shown here, or as part of another bluesky plan.

In [11]:
print_position(temperature)
RE(bps.mv(temperature, 25))
print_position(temperature)

inposition=True  position=25.841  setpoint=25.145
inposition=True  position=25.841  setpoint=25.000


`bps.mvr()` will make a *relative* move.  Decrease the temperature by `5`.

<b>Note</b> that `bps.mvr()` has set the new setpoint to exactly 5 below the
previous *readback* value (not from the previous *setpoint* value).

In [12]:
print_position(temperature)
RE(bps.mvr(temperature, -5))
print_position(temperature)

inposition=True  position=25.841  setpoint=25.000
inposition=True  position=21.435  setpoint=20.841


We can change the setpoint value directly.  But notice that the temperature is not inposition immediately.  This is because we asked for bluesky to wait *only* until setpoint changed, which happened almost instantly.

In [13]:
print_position(temperature)
RE(bps.mvr(temperature.setpoint, 5))
print_position(temperature)

inposition=True  position=21.435  setpoint=20.841
inposition=False  position=21.435  setpoint=25.841


We can measure the readback value (over time) by using `temperature` as a detector.  Here we use the `bp.count` plan, making 5 readings at 1 second intervals.  A data table is printed since this is one of the bluesky plans (`bp`) that create a [run](https://blueskyproject.io/bluesky/multi_run_plans.html#definition-of-a-run) which collects data.

<b>Tip</b>  If this cell is executed immediately after the preceding cell, then it will follow the readback as it approaches the new setpoint.

In [14]:
RE(bp.count([temperature], delay=1, num=5))



Transient Scan ID: 1     Time: 2023-12-24 16:43:15
Persistent Unique Scan ID: '9ff16964-06fb-40c9-ad9f-03127be51886'
New stream: 'primary'
+-----------+------------+-------------+
|   seq_num |       time | temperature |
+-----------+------------+-------------+
|         1 | 16:43:15.7 |    21.43473 |
|         2 | 16:43:16.7 |    23.36815 |
|         3 | 16:43:17.7 |    26.01955 |
|         4 | 16:43:18.7 |    26.75007 |
|         5 | 16:43:19.7 |    25.47070 |
+-----------+------------+-------------+
generator count ['9ff16964'] (scan num: 1)





('9ff16964-06fb-40c9-ad9f-03127be51886',)

To demonstrate the use of `temperature` as a positioner in a scan, we'll need another signal to use as a detector.  We'll create a simple ophyd Signal with a value that does not change.

In [15]:
from ophyd import Signal

det = Signal(name="det", value="123.45")

To see the temperature setpoint reported in the table, set its `kind` attribute to `"hinted"`.  Hinted attributes are shown (and plotted) when they are used as detectors.

In [16]:
temperature.setpoint.kind = "hinted"

Scan `det` vs. `temperature` in 5 steps from 20..40.  See how it is the
*setpoint* which is advanced in even steps.  The `bp.scan()` plan adjusts the
setpoint at each step, waits for the move to complete, then triggers and reads
the detectors.

In [17]:
RE(bp.scan([det], temperature, 20, 40, 5))



Transient Scan ID: 2     Time: 2023-12-24 16:43:20
Persistent Unique Scan ID: 'dfcb093e-85b0-4329-bdf7-4994bd9daa83'
New stream: 'primary'
+-----------+------------+-------------+----------------------+------------+
|   seq_num |       time | temperature | temperature_setpoint |        det |
+-----------+------------+-------------+----------------------+------------+
|         1 | 16:43:23.7 |    19.70829 |             20.00000 |        123 |
|         2 | 16:43:26.7 |    24.17618 |             25.00000 |        123 |
|         3 | 16:43:30.7 |    30.02489 |             30.00000 |        123 |
|         4 | 16:43:33.7 |    34.72389 |             35.00000 |        123 |
|         5 | 16:43:36.7 |    39.37125 |             40.00000 |        123 |
+-----------+------------+-------------+----------------------+------------+
generator scan ['dfcb093e'] (scan num: 2)





('dfcb093e-85b0-4329-bdf7-4994bd9daa83',)

`bp.rel_scan()` chooses its limits *relative* to the current position.  Here we scan from `-17` to `3`, *relative* to the current position.

In [18]:
print_position(temperature)
RE(bp.rel_scan([det], temperature, -17, 3, 5))

inposition=True  position=39.371  setpoint=40.000


Transient Scan ID: 3     Time: 2023-12-24 16:43:36
Persistent Unique Scan ID: 'aa441f52-feb8-48a8-b7d9-255caf979eff'
New stream: 'primary'
+-----------+------------+-------------+----------------------+------------+
|   seq_num |       time | temperature | temperature_setpoint |        det |
+-----------+------------+-------------+----------------------+------------+
|         1 | 16:43:44.7 |    23.04608 |             22.37125 |        123 |
|         2 | 16:43:46.7 |    27.33362 |             27.37125 |        123 |
|         3 | 16:43:48.7 |    31.95131 |             32.37125 |        123 |
|         4 | 16:43:50.7 |    37.00290 |             37.37125 |        123 |
|         5 | 16:43:53.7 |    42.25406 |             42.37125 |        123 |
+-----------+------------+-------------+----------------------+------------+
generator rel_scan ['aa441f52'] (scan num: 3)





('aa441f52-feb8-48a8-b7d9-255caf979eff',)

## SimulatedTemperatureController device

Combine the setup steps into a single ophyd Device to make a simulator.  Show the support code first:

In [19]:
%pycat ../../../bluesky/user/temperature_controller.py

"""
Create a simulated temperature controller device.

Example::

    temperature = SimulatedTemperatureController(
        "",
        name="temperature",
        swait_pv="gp:userCalc17",
        tolerance=1,
    )
    temperature.wait_for_connection()
    temperature.setup_controller(25, label="temperature controller")

.. note:: This code is used in the documentation.
   Leave it in the `user` directory.
"""

from apstools.devices import PVPositionerSoftDoneWithStop
from apstools.synApps import SwaitRecord
from ophyd import FormattedComponent as FC


class SimulatedTemperatureController(PVPositionerSoftDoneWithStop):
    """
    Simulated temperature controller with swait record.

    The swait record completes the feedback loop, computing
    the next simulated temperature.
    """

    loop = FC(SwaitRecord, "{swait_pv}", kind="config")

    def __init__(self, *args, swait_pv="", **kwargs):
        if len(swait_pv.strip()) == 0:
            raise ValueError("Must supply a value f

Adjust the Python module import path so this class can be used:

In [20]:
import pathlib, sys

path = pathlib.Path("../../..") / "bluesky" / "user"
sys.path.append(str(path))
from temperature_controller import SimulatedTemperatureController

Demonstrate the class by setting up a new temperature controller using a
different `swait` record.

In [21]:
t17 = SimulatedTemperatureController(
    "",
    name="t17",
    swait_pv="gp:userCalc17",
    tolerance=1,
)
t17.wait_for_connection()
t17.setup_controller(25, label="t17 controller")

Watch as the `t17` comes up to temperature.

In [22]:
t17.position
t0 = time.time()
while not t17.inposition:
    time.sleep(1)
    print_position(t17)

inposition=False  position=2.093  setpoint=25.000
inposition=False  position=4.874  setpoint=25.000
inposition=False  position=7.551  setpoint=25.000
inposition=False  position=10.213  setpoint=25.000
inposition=False  position=12.329  setpoint=25.000
inposition=False  position=13.530  setpoint=25.000
inposition=False  position=16.528  setpoint=25.000
inposition=False  position=18.948  setpoint=25.000
inposition=False  position=21.252  setpoint=25.000
inposition=False  position=23.597  setpoint=25.000
inposition=True  position=25.692  setpoint=25.000
